In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import numpy as np

In [2]:

model_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"

IMAGE_SHAPE = (224, 224)

layer = hub.KerasLayer(model_url, input_shape=IMAGE_SHAPE+(3,))
model = tf.keras.Sequential([layer])

2022-08-25 10:15:53.052825: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-25 10:15:53.053498: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


2022-08-25 10:15:53.820597: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-25 10:15:53.825504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-25 10:15:54.009930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [3]:
def extract(file):
    file = Image.open(file).convert('L').resize(IMAGE_SHAPE)
    #display(file)

    file = np.stack((file,)*3, axis=-1)

    file = np.array(file)/255.0

    embedding = model.predict(file[np.newaxis, ...])
    #print(embedding)
    vgg16_feature_np = np.array(embedding)
    flattended_feature = vgg16_feature_np.flatten()

    #print(len(flattended_feature))
    #print(flattended_feature)
    #print('-----------')
    return flattended_feature

In [4]:
jigap = extract('jigap01.jpeg')

2022-08-25 10:15:54.753457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 378ms/step


In [5]:
!pwd

/Users/iseongmin/workspaces/Image_similarity_practice


In [6]:
jigap

array([0.        , 0.        , 0.51531535, ..., 0.        , 0.        ,
       0.        ], dtype=float32)

In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [38]:
import os
for filename in (os.listdir("./sample_data")):
    print(filename)
    globals()['img_{}'.format(filename[4:6])] = extract('./sample_data/' + filename)


img_00.jpeg
1/1 [==============================] - 0s 21ms/step
img_01.jpeg
1/1 [==============================] - 0s 13ms/step
img_06.jpeg
1/1 [==============================] - 0s 13ms/step
img_07.jpeg
1/1 [==============================] - 0s 13ms/step
img_08.jpeg
1/1 [==============================] - 0s 13ms/step
img_04.jpeg
1/1 [==============================] - 0s 13ms/step
img_05.jpeg
1/1 [==============================] - 0s 14ms/step
img_02.jpeg
1/1 [==============================] - 0s 12ms/step
img_03.jpeg
1/1 [==============================] - 0s 13ms/step


In [60]:
from scipy.spatial import distance
metric = 'cosine'


cosineDistance = distance.cdist([img_03], [img_05], metric)[0]
print(cosineDistance)
print("the distance between img_01 and img_02 is {}".format(cosineDistance)) 

[0.44967505]
the distance between img_01 and img_02 is [0.44967505]


In [77]:
import pandas as pd
import numpy as np

In [88]:

img_list = [img_00, img_01, img_02, img_03, img_04, img_05, img_06, img_07, img_08]

for img in img_list:
    globals()['{}_sim'.format(img)] = list()
    
    for i in img_list:
        cosineDistance = distance.cdist([img], [i], metric)[0]
        globals()['{}_sim'.format(img)].append(cosineDistance)
    print(np.argmin(globals()['{}_sim'.format(img)]))
#     print('{} / {} / {} \n{} / {} / {} \n {} / {} / {} '.format(globals()['{}_sim'.format(img)]))
#     print('.3f' % element for element in globals()['{}_sim'.format(img)])
#     print('{}'.format(tuple(globals()['{}_sim'.format(img)])))
    print(globals()['{}_sim'.format(img)], '\n')
    


0
[array([0.]), array([0.70494081]), array([0.45787527]), array([0.56707659]), array([0.54926301]), array([0.49123411]), array([0.40538666]), array([0.49611175]), array([0.31715296])] 

1
[array([0.70494081]), array([0.]), array([0.46810029]), array([0.30402393]), array([0.39206186]), array([0.51535129]), array([0.50855588]), array([0.63459254]), array([0.68279351])] 

2
[array([0.45787527]), array([0.46810029]), array([0.]), array([0.40902797]), array([0.39185411]), array([0.42761725]), array([0.23528955]), array([0.59609759]), array([0.50410495])] 

3
[array([0.56707659]), array([0.30402393]), array([0.40902797]), array([0.]), array([0.30064206]), array([0.44967505]), array([0.37602147]), array([0.61838416]), array([0.57447904])] 

4
[array([0.54926301]), array([0.39206186]), array([0.39185411]), array([0.30064206]), array([0.]), array([0.45795692]), array([0.39909415]), array([0.55109169]), array([0.59927964])] 

5
[array([0.49123411]), array([0.51535129]), array([0.42761725]), arra

In [91]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="2hZItvoco7YwHNaFgCD4")
project = rf.workspace("dsproject2jo").project("lost_article")
dataset = project.version(1).download("yolov5")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.5/145.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 36.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for roboflow: filename=roboflow-0.2.14-py3-none-any.whl size=24818 sha256=a1788266fc7427a6f40d354bd6244791e8584b17cc24461e07c3bc9364a00310
  Stored in directory: /Users/iseongmin/Library/Caches/pip/wheels/90

Extracting Dataset Version Zip to lost_article-1 in yolov5pytorch:: 100%|█| 30/3


In [92]:
dataset